In [ ]:
pip install xvfbwrapper

In [ ]:
pip install pyvirtualdisplay

In [ ]:
!pip install requests beautifulsoup4 python-dateutil

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import pickle
import os
from collections import Counter
from datetime import datetime, timedelta
from dateutil import parser
from urllib.parse import urljoin
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pyvirtualdisplay import Display
from requests.structures import CaseInsensitiveDict
from datetime import datetime
from http.cookiejar import CookieJar
from collections import CaseInsensitiveDict



In [ ]:
with open("tokens_relevantes.txt", "r") as f:
    tokens_relevantes = [line.strip() for line in f]

In [ ]:
with open("tokens_irrelevantes.txt", "r") as f:
    tokens_irrelevantes = [line.strip() for line in f]

# Tokenizando notícias do Estadão, CNN e Crusoé

In [ ]:
tokens_inuteis = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k',
                  'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
                  'w', 'x', 'y', 'z', 'º', 'aut', 'newsletters', 'pop',
                  'podcasts', 'brasileirão', 'nemo', 'enim', 'ipsam',
                  'voluptatem', 'quia', 'voluptas', 'sit', 'aspernatur',
                 'odit', 'fugit', 'newsletter', 'diz', 'veja', 'leia',
                 'news', 'you', 'tube', 'youtube', 'linkedin', 'in',
                  'linked', 'pinterest', 'metacast', 'cable', 'network',
                 'horóscopo', 'quadrinhos', 'broadcast', 'ví', 'assinantes',
                 'assine', 'assinante']

In [ ]:
def tokenizar_noticias_relevantes(urls_relevantes, email=None, senha=None):
    tokens_gerados = []
    tokens_sem_stopwords = [] # declaração fora do loop

    for url in urls_relevantes:
        # Faz a autenticação no paywall se necessário
        sessao = requests.Session()
        if 'oantagonista' in url or 'crusoe' in url:
            if email is None or senha is None:
                print("Email e senha são necessários para acessar o paywall.")
                return None
            resposta_login = sessao.post("https://crusoe.uol.com.br/ws/uid/3afde8142c87d233be5421ac3be7385e", json={"username": email, "password": senha})
        elif 'estadao' in url:
            if email is None or senha is None:
                print("Email e senha são necessários para acessar o paywall.")
                return None
            resposta_login = sessao.post("https://acesso.estadao.com.br/login", data={"username": email, "password": senha})
        else:
            resposta_login = None

        if resposta_login is not None and resposta_login.status_code != 200:
            print("Não foi possível fazer login no paywall.")
            return None

        # Faz o download do HTML da página da notícia
        resposta = sessao.get(url)
        if resposta.status_code != 200:
            print("Não foi possível acessar a página da notícia.")
            continue
        html_pagina = resposta.content

        # Extrai o texto do conteúdo HTML
        soup = BeautifulSoup(html_pagina, 'html.parser')
        texto_noticia = soup.get_text()

        # Remove quebras de linha, espaços em branco extras e caracteres especiais
        texto_noticia = re.sub('\s+', ' ', texto_noticia).strip()
        # Separa palavras que estão juntas
        texto_noticia = re.sub('(?<=\w)(?=[A-Z])', ' ', texto_noticia)
        texto_noticia = re.sub('(?<=\w)(?=\d)', ' ', texto_noticia)
        texto_noticia = re.sub('(?<=\d)(?=\w)', ' ', texto_noticia)

        # Tokeniza o texto da notícia
        tokens = nltk.word_tokenize(texto_noticia)

        stop_words = set(stopwords.words('portuguese'))

        for token in tokens:
            if token.lower() not in stop_words and token.isalpha():
                tokens_sem_stopwords.append(token.lower())
                
         # Remove tokens inúteis da lista        
        tokens_gerados = [t for t in tokens_gerados if t not in tokens_inuteis]

        # Adiciona os novos tokens à lista de tokens relevantes
        tokens_gerados.extend(tokens_sem_stopwords)

    # Salva a lista de tokens gerados em um arquivo
    with open("tokens_relevantes.txt", "a") as f:
        f.write("\n".join(tokens_sem_stopwords) + "\n")
        
    return tokens_gerados

In [ ]:
def tokenizar_noticias_irrelevantes(urls_irrelevantes, email=None, senha=None):
    tokens_gerados = []
    tokens_sem_stopwords = [] # declaração fora do loop

    for url in urls_irrelevantes:
        # Faz a autenticação no paywall se necessário
        sessao = requests.Session()
        if 'oantagonista' in url or 'crusoe' in url:
            if email is None or senha is None:
                print("Email e senha são necessários para acessar o paywall.")
                return None
            resposta_login = sessao.post("https://crusoe.uol.com.br/ws/uid/3afde8142c87d233be5421ac3be7385e", json={"username": email, "password": senha})
        elif 'estadao' in url:
            if email is None or senha is None:
                print("Email e senha são necessários para acessar o paywall.")
                return None
            resposta_login = sessao.post("https://acesso.estadao.com.br/login", data={"username": email, "password": senha})
        else:
            resposta_login = None

        if resposta_login is not None and resposta_login.status_code != 200:
            print("Não foi possível fazer login no paywall.")
            return None

        # Faz o download do HTML da página da notícia
        resposta = sessao.get(url)
        if resposta.status_code != 200:
            print("Não foi possível acessar a página da notícia.")
            continue
        html_pagina = resposta.content

        # Extrai o texto do conteúdo HTML
        soup = BeautifulSoup(html_pagina, 'html.parser')
        texto_noticia = soup.get_text()

        # Remove quebras de linha, espaços em branco extras e caracteres especiais
        texto_noticia = re.sub('\s+', ' ', texto_noticia).strip()
        # Separa palavras que estão juntas
        texto_noticia = re.sub('(?<=\w)(?=[A-Z])', ' ', texto_noticia)
        texto_noticia = re.sub('(?<=\w)(?=\d)', ' ', texto_noticia)
        texto_noticia = re.sub('(?<=\d)(?=\w)', ' ', texto_noticia)

        # Tokeniza o texto da notícia
        tokens = nltk.word_tokenize(texto_noticia)

        stop_words = set(stopwords.words('portuguese'))

        for token in tokens:
            if token.lower() not in stop_words and token.isalpha():
                tokens_sem_stopwords.append(token.lower())
                
        # Remove tokens inúteis da lista        
        tokens_gerados = [t for t in tokens_gerados if t not in tokens_inuteis]

        # Adiciona os novos tokens à lista de tokens irrelevantes
        tokens_gerados.extend(tokens_sem_stopwords)

    # Salva a lista de tokens gerados em um arquivo
    with open("tokens_irrelevantes.txt", "a") as f:
        f.write("\n".join(tokens_sem_stopwords) + "\n")
        
    return tokens_gerados

In [ ]:
urls_relevantes = ['https://www.cnnbrasil.com.br/politica/a-pf-ex-ministro-goncalves-dias-diz-que-nao-foi-omisso-e-que-nao-tem-responsabilidade-sobre-atos-de-8-1/', 'https://www.cnnbrasil.com.br/politica/pl-define-parte-dos-nomes-para-cpi-dos-atos-de-8-de-janeiro/', 'https://www.cnnbrasil.com.br/politica/governo-avanca-para-ter-maioria-na-cpi-do-8-1-e-define-alvos-veja-nomes/', 'https://www.cnnbrasil.com.br/politica/governo-avanca-para-ter-maioria-na-cpi-do-8-1-e-define-alvos-veja-nomes/', 'https://www.cnnbrasil.com.br/politica/moraes-determina-quebra-de-sigilo-de-todas-as-imagens-do-8-de-janeiro/', 'https://www.cnnbrasil.com.br/politica/mucio-diz-lamentar-queda-de-goncalves-dias-um-dos-mais-fieis-amigos-de-lula/', 'https://www.cnnbrasil.com.br/politica/oposicao-defende-que-pgr-denuncie-goncalves-dias-por-prevaricacao/', 'https://www.cnnbrasil.com.br/politica/camara-dos-deputados-deve-votar-projeto-das-fake-news-nesta-semana/', 'https://www.cnnbrasil.com.br/politica/deputados-debatem-sobre-mudanca-de-postura-do-governo-em-relacao-a-cpi-de-8-de-janeiro/', 'https://www.cnnbrasil.com.br/politica/nao-adianta-discutir-com-big-techs-antes-de-a-camara-analisar-texto-diz-relator-do-pl-das-fake-news-no-senado/', 'https://www.cnnbrasil.com.br/politica/apos-determinacao-do-stf-gsi-libera-acesso-a-imagens-do-8-de-janeiro-mas-site-da-pane/', 'https://www.cnnbrasil.com.br/politica/a-pf-ex-ministro-cita-apagao-geral-e-falta-de-condicoes-de-efetuar-prisoes-sozinho-em-8-1/', 'https://www.cnnbrasil.com.br/politica/abin-nao-informou-sobre-riscos-de-8-1-com-antecedencia-diz-ex-ministro-a-pf/', 'https://www.cnnbrasil.com.br/politica/a-pf-ex-ministro-diz-que-se-tivesse-visto-teria-prendido-major-que-deu-agua-a-invasores-do-planalto/', 'https://www.cnnbrasil.com.br/politica/alckmin-diz-que-goncalves-dias-fez-o-correto-em-pedir-demissao/', 'https://www.cnnbrasil.com.br/politica/policia-federal-vai-ouvir-mais-nove-militares-do-gsi-neste-domingo/', 'https://www.estadao.com.br/politica/mariana-carneiro/criticas-de-favaro-ao-mst-foram-combinadas-com-lula/', 'https://www.estadao.com.br/politica/mariana-carneiro/deputados-do-pt-insistem-em-alterar-limite-para-investimentos-do-novo-marco-fiscal/', 'https://www.estadao.com.br/politica/pl-das-fake-news-mais-de-100-deputados-e-big-techs-pressionam-para-adiar-votacao-do-projeto/', 'https://www.estadao.com.br/politica/mariana-carneiro/tarcisio-e-o-favorito-a-substituir-bolsonaro-em-caso-de-inelegibilidade/', 'https://www.estadao.com.br/politica/lula-diz-que-definicao-sobre-gsi-fica-para-volta-ao-brasil-e-que-congresso-decide-sobre-cpmi/', 'https://www.estadao.com.br/politica/momentos-de-tensao-entre-ministros-do-lula-no-dia-da-invasao-ao-planalto-veja-video/', 'https://www.estadao.com.br/politica/mariana-carneiro/governo-deixa-de-tratar-impeachment-de-dilma-como-golpe-em-registros-oficiais-apos-acao-na-justica/', 'https://www.estadao.com.br/politica/mariana-carneiro/governo-diz-apoiar-cpmi-do-8-de-janeiro-mas-parlamentares-da-base-se-negam-a-assinar-requerimento/', 'https://www.estadao.com.br/politica/mariana-carneiro/ataques-de-salles-a-nunes-desagradam-ala-do-pl-ligada-a-valdemar-costa-neto/', 'https://www.estadao.com.br/politica/bolsonaristas-pressionam-tarcisio-por-mudancas-na-cultura-e-saida-de-secretaria/', 'https://www.estadao.com.br/politica/unica-camera-com-audio-do-planalto-mostra-tiros-bombas-e-golpistas-tomando-palacio-veja-video/', 'https://www.estadao.com.br/politica/veja-o-passo-a-passo-de-ministro-do-gsi-no-planalto-do-susto-ao-ver-golpistas-a-encontro-com-lula/', 'https://www.estadao.com.br/politica/colunas/militares-que-aparecem-em-imagens-acompanharam-bolsonaro-em-compromisso-eleitoral-e-motociatas/', 'https://crusoe.uol.com.br/diario/governo-quer-tirar-andre-fernandes-da-cpmi-de-8-de-janeiro/', 'https://crusoe.uol.com.br/diario/exclusivo-governo-lula-gasta-13-vezes-mais-em-propaganda-que-bolsonaro-no-inicio-de-mandato/', 'https://crusoe.uol.com.br/diario/socialistas-demonstram-insatisfacao-com-ingresso-do-psb-em-blocao-de-lira/', ]

In [ ]:
urls_irrelevantes = ['https://www.cnnbrasil.com.br/politica/veja-a-integra-do-depoimento-do-ex-ministro-goncalves-dias-a-pf/', 'https://www.cnnbrasil.com.br/politica/militares-do-gsi-que-aparecem-em-imagens-do-8-de-janeiro-prestam-depoimento-a-pf/', 'https://www.cnnbrasil.com.br/politica/anderson-torres-piora-e-defesa-pede-avaliacao-psicologica/', 'https://www.cnnbrasil.com.br/politica/apos-divulgacao-de-imagens-do-8-de-janeiro-major-que-deu-agua-a-manifestantes-e-mais-oito-militares-do-gsi-depoem-a-pf/', 'https://www.cnnbrasil.com.br/politica/ministra-anielle-franco-discute-discriminacao-racial-em-viagem-oficial-a-portugal/', 'https://www.cnnbrasil.com.br/politica/janja-e-vista-fazendo-compras-em-loja-de-luxo-em-portugal/', 'https://www.cnnbrasil.com.br/politica/lula-participa-de-cerimonia-com-presidente-de-portugal-e-homenagem-a-camoes/', 'https://www.cnnbrasil.com.br/politica/lula-pede-desculpas-sobre-fala-em-que-diz-que-pessoas-com-transtornos-mentais-tem-desequilibrio-de-parafuso/', 'https://www.cnnbrasil.com.br/politica/policia-federal-vai-ouvir-mais-nove-militares-do-gsi-neste-domingo/','https://www.cnnbrasil.com.br/politica/ao-jornal-valor-ministro-interino-do-gsi-critica-cnn/', 'https://www.estadao.com.br/politica/legis-ativo/legis-ativo-resumo-da-semana-discussao-envolvendo-os-deputados-federais-eduardo-bolsonaro-pl-sp-e-dionilso-marcon-pt-rs-comissao-de-etica-arcabouco-fiscal-instalacao-de-cpmi-piso-salarial-d/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/habitos-manias/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/governo-desiste-de-tributar-produtos-importados-por-pessoas-fisicas/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/ciencia-a-servico-do-mal-2/', 'https://www.estadao.com.br/politica/lula-enfrenta-questionamentos-da-classe-politica-em-meio-a-uma-agenda-de-aproximacao-com-portugal/', 'https://www.estadao.com.br/politica/janja-publica-foto-de-lula-com-gravata-nova-apos-repercussao-em-loja-de-luxo/', 'https://www.estadao.com.br/politica/lula-volta-a-dizer-em-portugal-que-russia-e-ucrania-nao-querem-parar-guerra-e-defende-terceira-via/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/comissionado-governo-acre-exoneracao-golpe-vazamento-conversas-intimas/', 'https://www.estadao.com.br/politica/mst-inicia-desocupacao-de-area-de-pesquisa-da-embrapa-invadida-em-pernambuco/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/pf-depoimentos-servidores-gsi-8-janeiro/', 'https://www.estadao.com.br/politica/imagens-mostram-golpistas-acolhidos-por-militar-no-planalto-e-tentativa-de-roubo-a-caixa-eletronico/', 'https://www.estadao.com.br/politica/eliane-cantanhede/a-cpi-vem-ai-e-lula-alimenta-o-bolsonarismo-que-e-craque-em-inverter-a-verdade/', 'https://www.estadao.com.br/politica/j-r-guzzo/imagens-de-um-pais-sem-lei/', 'https://www.estadao.com.br/politica/congresso-manobra-e-usa-r-287-mi-de-emenda-pix-durante-campanha-eleitoral/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/o-heraldico-brio-do-paulistano/', 'https://www.estadao.com.br/politica/mariana-carneiro/sindicalistas-nao-se-entendem-sobre-regulamentacao-de-trabalho-por-aplicativo/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/acessibilidade-vagas-de-garagem-para-pcds-e-idosos/', 'https://www.estadao.com.br/politica/governo-zema-chama-inconfidencia-de-golpe-e-diz-que-tiradentes-confessou-crimes/', 'https://www.estadao.com.br/politica/lula-ameniza-discurso-e-diz-que-nao-pediu-que-europa-e-eua-mudassem-postura-sobre-guerra-na-ucrania/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/pf-stf-4-mil-horas-gravacao-atos-golpistas-pericia/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/video-choque-pm-manifestantes-planalto-16-minutos/', 'https://www.estadao.com.br/politica/dedo-para-camera-destruicao-de-raio-x-e-roubo-de-estatua-videos-mostram-golpistas-em-acao-no-81/', 'https://www.estadao.com.br/politica/mst-deixa-area-da-embrapa-em-pernambuco-e-faz-tres-novas-invasoes-de-fazendas-na-bahia/', 'https://www.estadao.com.br/politica/videos-mostram-falhas-de-policiais-em-remocao-de-golpistas-do-planalto-em-81/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/avaliacao-psicologica-anderson-torres-preso-100-dias/', 'https://www.estadao.com.br/politica/blog-do-fausto-macedo/atos-golpistas-servidores-gsi-inteligencia-risco-violencia-efetivo-insuficiente/', 'https://crusoe.uol.com.br/diario/como-a-oposicao-argentina-recebeu-a-noticia-que-fernandez-desistiu-de-tentar-reeleicao/', 'https://crusoe.uol.com.br/diario/trumpista-perde-desafio-de-fraude-eleitoral-e-tera-de-pagar-us-5-milhoes/', 'https://crusoe.uol.com.br/diario/latitude22-eleicoes-na-argentina-mendoexit-e-vinho-amarelo/', 'https://crusoe.uol.com.br/diario/a-reunificacao-da-alemanha-e-a-construcao-de-brasilia/', 'https://crusoe.uol.com.br/diario/a-seca-de-votos-que-levou-alberto-fernandez-a-desistir-da-reeleicao/', 'https://crusoe.uol.com.br/diario/os-numeros-da-ebc/', 'https://crusoe.uol.com.br/diario/os-chineses-estao-de-olho-no-mercado-de-defesa-do-brasil/', 'https://crusoe.uol.com.br/diario/chocolate-com-mais-pimenta/', 'https://crusoe.uol.com.br/diario/mulheres-precisam-de-leis-para-faltar-ao-trabalho-quando-sentirem-colicas/']

In [ ]:
tokenizar_noticias_relevantes(urls_relevantes, 'email', 'senha')

In [ ]:
tokenizar_noticias_irrelevantes(urls_irrelevantes, 'email', 'senha')

In [ ]:
len(tokens_relevantes)

In [ ]:
len(tokens_irrelevantes)

In [ ]:
tokens_relevantes = [t for t in tokens_relevantes if t not in tokens_inuteis]

In [ ]:
tokens_irrelevantes = [t for t in tokens_irrelevantes if t not in tokens_inuteis]

In [ ]:
contagem_palavras_relevantes = Counter(tokens_relevantes)
contagem_palavras_relevantes.most_common()

In [ ]:
contagem_palavras_irrelevantes = Counter(tokens_irrelevantes)
contagem_palavras_irrelevantes.most_common()

# Buscando notícias de forma automática nos jornais

In [ ]:
texto_noticia = "Um estudo recente descobriu que a prática regular de exercícios físicos pode ajudar a prevenir uma série de doenças crônicas, como diabetes, hipertensão e doenças cardíacas. Além disso, a atividade física também pode ter efeitos positivos no humor e na saúde mental, ajudando a reduzir o estresse e a ansiedade. O estudo destaca a importância de incorporar exercícios físicos na rotina diária para melhorar a qualidade de vida e prevenir doenças."

In [ ]:
def pontuacao_noticia(texto_noticia, tokens_relevantes, tokens_irrelevantes):
    # Converte texto para minúsculas
    texto_noticia = texto_noticia.lower()

    # Remove caracteres indesejados
    texto_noticia = re.sub('[^a-zA-Z0-9 \n\.]', '', texto_noticia)

    # Remove quebras de linha, espaços em branco extras e caracteres especiais
    texto_noticia = re.sub('\s+', ' ', texto_noticia).strip()
    # Separa palavras que estão juntas
    texto_noticia = re.sub('(?<=\w)(?=[A-Z])', ' ', texto_noticia)
    texto_noticia = re.sub('(?<=\w)(?=\d)', ' ', texto_noticia)
    texto_noticia = re.sub('(?<=\d)(?=\w)', ' ', texto_noticia)

    # Tokeniza o texto
    tokens = texto_noticia.split()

    # Remove tokens irrelevantes e atribui penalidade
    relevant_tokens = [token.lower() for token in tokens_relevantes]
    irrelevant_tokens = [token.lower() for token in tokens_irrelevantes]
    tokens = [token for token in tokens if token not in irrelevant_tokens]

    # Calcula pontuação
    pontuacao = 0
    for token in tokens:
        if token in relevant_tokens:
            pontuacao += 1
        elif token in irrelevant_tokens:
            pontuacao -= 1

    return pontuacao

In [ ]:
pontuacao_noticia(texto_noticia, tokens_relevantes, tokens_irrelevantes)

In [ ]:
def buscar_noticias_estadao(date):
    login_url = 'https://acesso.estadao.com.br/login'
    noticias_url = 'https://www.estadao.com.br/politica'
    headers = {
    "Cookie": "sb=QdnQYKeF9IFsdGcCkP7Pog1z; datr=CL3RYBD4JmSVPMBXFGZzvKay; c_user=100055367296182; xs=37:uTmoHWqKkbWKaw:2:1679953928:-1:-1; fr=0FMF8CWHuAoJUIvaO.AWUqq_kKGVKz2uHFo497Z4B8UYU.Bj78Gm.3C.AAA.0.0.BkRb3g.AWWGh39N8qM"
    }
    session = requests.Session()

    # Realiza login para acessar as notícias
    response = session.get(login_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    csrf_token = soup.find('input', {'name': 'CSRF-TOKEN'})
    if csrf_token is None:
        raise Exception('Token CSRF não encontrado')
    csrf_token = csrf_token.get('value')

    login_data = {
        'username': 'email',
        'password': 'senha',
        'CSRF-TOKEN': csrf_token,
    }

    response = session.post(login_url, data=login_data)
    response.raise_for_status()

    # Acessa a página de notícias
    response = session.get(noticias_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontra as notícias relevantes da data especificada
    noticias = []
    links_adicionados = []
    lista_noticias_com_pontuacao = []
    for article in soup.find_all('article'):
        timestamp = article.find('time').get('datetime')
        date_noticia = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S.%fZ').date()
        if date_noticia == date:
            titulo = article.find('h3').get_text().strip()
            descricao = article.find('p').get_text().strip()
            link = article.find('a').get('href')
            if link:
                link = 'https://www.estadao.com.br' + link
                # Verifica se o link já foi adicionado
                if link not in links_adicionados:
                    # Adiciona o link na lista de links adicionados
                    links_adicionados.append(link)
                    # Calcula a pontuação da notícia
                    texto_noticia = requests.get(link).text
                    pontuacao = pontuacao_noticia(texto_noticia, tokens_relevantes, tokens_irrelevantes)
                    # Adiciona a notícia e sua pontuação na lista de notícias
                    lista_noticias_com_pontuacao.append({'titulo': titulo, 'link': link, 'pontuacao': pontuacao})

    noticias_ordenadas = sorted(lista_noticias_com_pontuacao, key=lambda noticia: noticia['pontuacao'], reverse=True)

    return noticias_ordenadas

In [ ]:
buscar_noticias_estadao(data, 'email', 'senha')

In [ ]:
def buscar_noticias_estadao(date, username, password):
    login_url = 'https://acesso.estadao.com.br/login'
    noticias_url = 'https://www.estadao.com.br/politica'

    session = requests.Session()

    # Realiza login para acessar as notícias
    response = session.get(login_url)
    

    login_data = {
        'username': username,
        'password': password,
    }

    response = session.post(login_url, data=login_data)
    response.raise_for_status()

    # Acessa a página de notícias
    response = session.get(noticias_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontra as notícias relevantes da data especificada
    noticias = []
    links_adicionados = []
    lista_noticias_com_pontuacao = []
    for article in soup.find_all('article'):
        timestamp = article.find('time').get('datetime')
        date_noticia = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S.%fZ').date()
        if date_noticia == date:
            titulo = article.find('h3').get_text().strip()
            descricao = article.find('p').get_text().strip()
            link = article.find('a').get('href')
            if link:
                link = 'https://www.estadao.com.br' + link
                print(f'Processando notícia: {titulo} ({link})')
                # Verifica se o link já foi adicionado
                if link not in links_adicionados:
                    # Adiciona o link na lista de links adicionados
                    links_adicionados.append(link)
                    # Calcula a pontuação da notícia
                    texto_noticia = requests.get(link).text
                    pontuacao = pontuacao_noticia(texto_noticia, tokens_relevantes, tokens_irrelevantes)
                    print(f'Pontuação da notícia: {pontuacao}')
                    # Adiciona a notícia e sua pontuação na lista de notícias
                    lista_noticias_com_pontuacao.append({'titulo': titulo, 'link': link, 'pontuacao': pontuacao})

    noticias_ordenadas = sorted(lista_noticias_com_pontuacao, key=lambda noticia: noticia['pontuacao'], reverse=True)
    print(f'Total de notícias encontradas: {len(noticias_ordenadas)}')

    return noticias_ordenadas
